In [7]:
import os
import subprocess
import collections

import climatools.clirad.info as cliradinfo

In [17]:
CLIRADLW_BANDS = cliradinfo.wavenumber_bands(region='lw')

In [18]:
DIR_SRC = os.path.join('/chia_cluster/home/jackyu/radiation/crd',
                       'LW/src',
                       'lblnew_-_nref_-_autoabsth_klin_-_gasc_kdesc')

In [36]:
params_1 = {
    'molecule': 'h2o',
    'band': 8,
    'ref_pts': [(1, 250), (10, 250), (500, 250)],
    'ng_refs': [3, 2, 3],
    'wgt': [(.55, .55, .85), (.85, .85), (0, .3, .55)],
    'option_wgt_flux': 2,
    'option_wgt_k': 1,
    'klin': 0,
    'commitnumber': 'a1423',
    'atmpro': 'saw'
}

params_2 = {
    'molecule': 'h2o',
    'band': 6,
    'ref_pts': [(600, 250), ],
    'ng_refs': [4,],
    'wgt': [(.55, .6, .65, .95)],
    'option_wgt_flux': 2,
    'option_wgt_k': 1,
    'klin': 5e-25,
    'commitnumber': 'a3d1611',
    'atmpro': 'mls'
}

In [37]:
def enter_input_params(path_lblnew, params=None):
    pass

In [38]:
def get_dir_case(params):
    '''
    Returns the absolute path of the directory in which 
    to run the case with input parameters `params`
    '''   
    template = os.path.join(
        '/chia_cluster/home/jackyu/radiation/crd',
        'LW/examples',
        'separate_g_groups',
        'study__lblnew_g1_threshold',
        '{molecule}',
        'band{band:02d}_wn_{vmin:d}_{vmax:d}',
        'ng_{ng:d}',
        'g_ascending_k_descending',
        'refPTs_{refPTs}',
        'ng_refs_{ng_refs}',
        'getabsth_{getabsth}',
        'absth_{absth}',
        'wgt_{wgt}',
        'wgt_flux_{option_wgt_flux}',
        'wgt_k_{option_wgt_k}',
        'klin_{klin}',
        'crd_{commitnumber}',
        'atmpro_{atmpro}')
    
    nref = len(params['ng_refs'])
    vmin, vmax = clirad_bands[params['band']][0]
    ng = sum(params['ng_refs'])
    refPTs = '__'.join(['P_{}_T_{}'.format(*pt) 
                        for pt in params['ref_pts']])
    ng_refs = '__'.join([str(n) for n in params['ng_refs']])
    getabsth = '__'.join(['auto' for _ in range(nref)])
    absth = '__'.join(['dlogN_uniform' for _ in range(nref)])
    wgt = '__'.join(['_'.join([str(w) for w in wgt_ref]) 
                     for wgt_ref in params['wgt']])
    klin = 'none' if params['klin'] == 0 else params['klin']
    
    return template.format(molecule=params['molecule'],
                    band=params['band'], vmin=vmin, vmax=vmax,
                    ng=ng,
                    refPTs=refPTs,
                    ng_refs=ng_refs,
                    getabsth=getabsth,
                    absth=absth,
                    wgt=wgt,
                    option_wgt_flux=params['option_wgt_flux'],
                    option_wgt_k=params['option_wgt_k'],
                    klin=klin,
                    commitnumber=params['commitnumber'],
                    atmpro=params['atmpro'])

In [74]:
def run_cases(cases_params=None):
    
    procs = []
    for params in cases_params:
        dir_case = get_dir_case(params)
        
        print(dir_case)
        
        try:
            os.makedirs(dir_case)
        except FileExistsError:
            print(params, 'This case already exists.')
            print()
            procs.append(None)
            continue
            
        try:
            os.chdir(dir_case)
            assert os.system('cp {}/*.f .'.format(dir_src)) == 0
        except AssertionError:
            print('Problem copying source code to case directory for '
                  'case',
                  dir_case)
            print()
            procs.append(None)
            continue
            
        
        fname_code = 'lblnew.f'
        
        os.chdir(dir_case)
        enter_input_params(fname_code, params=params)
        
        try:
            os.chdir(dir_case)
            os.system('ifort -g -traceback -fpe0 {} -o lblnew.exe'.format(fname_code))
            assert os.path.exists('lblnew.exe') == True
        except AssertionError:
            print('Problem compiling source code for case',
                  params)
            print()
            procs.append(None)
            continue
        
        proc = subprocess.Popen(['./lblnew.exe'], stdout=subprocess.PIPE)
        
        procs.append(proc)
    
    print()
        
    return procs

In [75]:
cases_params = [params_1, params_2]

procs = run_cases(cases_params)

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band08_wn_1380_1900/ng_8/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_3__2__3/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/wgt_0.55_0.55_0.85__0.85_0.85__0_0.3_0.55/wgt_flux_2/wgt_k_1/klin_none/crd_a1423/atmpro_saw
{'molecule': 'h2o', 'band': 8, 'ref_pts': [(1, 250), (10, 250), (500, 250)], 'ng_refs': [3, 2, 3], 'wgt': [(0.55, 0.55, 0.85), (0.85, 0.85), (0, 0.3, 0.55)], 'option_wgt_flux': 2, 'option_wgt_k': 1, 'klin': 0, 'commitnumber': 'a1423', 'atmpro': 'saw'} This case already exists.

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band06_wn_1100_1215/ng_4/g_ascending_k_descending/refPTs_P_600_T_250/ng_refs_4/getabsth_auto/absth_dlogN_uniform/wgt_0.55_0.6_0.65_0.95/wgt_flux_2/wgt_k_1/klin_5e-25/crd_a3d1611/atmpro_mls



In [72]:
import shutil

[shutil.rmtree(get_dir_case(params)) for params in cases_params]


[None, None]

In [78]:
out, err = procs[1].communicate()

In [79]:
[proc.poll() for proc in procs]

AttributeError: 'NoneType' object has no attribute 'poll'

In [85]:
print(out.decode('utf-8'))

 iref           1
 klims  0.000000000000000E+000  0.000000000000000E+000
 ng_tosort           6
 kmin, kmax  3.094901839238028E-028  2.230675024591008E-018
 absth(1:ng_tosort, iref)  5.075413149326968E-020  1.154799258179001E-021
  2.627493146774898E-023  5.978285998585982E-025  1.360228228368852E-026
  3.094901839238020E-028
 iref           2
 klims  5.075413149326968E-020  0.000000000000000E+000
 ng_tosort           5
 kmin, kmax  1.858499373972203E-025  1.094189439668947E-019
 absth(1:ng_tosort, iref)  7.675496871279285E-021  5.384191263886007E-022
  3.776891066763531E-023  2.649405533915330E-024  1.858499373972204E-025
 absth  5.075413149326968E-020  1.154799258179001E-021  2.627493146774898E-023
  5.978285998585982E-025  1.360228228368852E-026  3.094901839238020E-028
  7.675496871279285E-021  5.384191263886007E-022  3.776891066763531E-023
  2.649405533915330E-024  1.858499373972204E-025  0.000000000000000E+000
 abscom  5.075413149326968E-020  7.675496871279285E-021  5.384191263886

In [87]:
1 + 1
None

In [ ]:
[proc.communicate() for proc in procs]